In [26]:
import pandas as pd
import numpy as np

df = pd.read_csv("0505_ENG_Korean.csv")

In [27]:
drama_titles = df.groupby('title',as_index=False).count().title
drama_titles

0                          18어게인
1                  365:운명을거스르는1년
2                       60일지정생존자
3                           D.P.
4     검색어를입력하세요WWW
                 ...            
87                  하늘에서내리는일억개의별
88                          하이에나
89                      한번다녀왔습니다
90                          해피니스
91                         호텔델루나
Name: title, Length: 92, dtype: object

In [28]:
reviews = df.reviews.to_list()
sentences = ",".join(reviews)
reviews_by_drama = []
for title in titles:
    rev_by_drama = df[df.title == title].reviews
    reviews_by_drama.append(','.join(rev_by_drama))

In [29]:
len(reviews)

27699

# 필요 함수들

In [ ]:
# pip install contextualized_topic_models

In [ ]:
# pip install tensorflow

In [33]:
import re
import pandas as pd
import numpy as np
import string
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import nltk
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
# from contextualized_topic_models.models.ctm import CombinedTM
# from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
# from konlpy.tag import Okt
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words = set(stopwords.words('english'))
stop_words.update(('\r\n', '\n\r', 'ever', 'much', 'look', 'squid', 'show', 'thing', "i've", 'anything', 'something', "show's",
                   'www', 'soompi' 'com', 'instagram', 'youtube', 'https', 'mydramalist', 'twitter', 'episode', 'comment', 'scene',
                   'version', "he's", 'gonna', 'series', 'watch', 'everything', 'something', "can't", 'list', 'dramas', 'drama',
                   'wait', 'preview', 'someone', 'everyone', 'dont', 'think', 'season', 'anyone', 'something', 'anything', 'nothing', 'world',
                   'status', 'week', 'name', 'cause', 'time', 'en', 'org', 'wikipedia', 'wiki', 'pbs', 'twimg', 'year', 'point', 'please', 'today',
                   'haha', 'case', 'guess', 'reason', 'person', 'moment', 'sense', 'kinda', 'part', 'movie', 'school', 'start', 'work', 'lead', 'kind',
                   'rate', 'rating', 'rate', 'men', 'example', 'idea', 'half', 'review', 'genre', 'side', "that's", "they're", 'till', 'tell', 'phone',
                   'section', 'number', 'company', 'line', "there's", 'male', 'team', 'rating', 'baby', 'course', 'care', 'cute', 'question', 'help', 'group',
                   'hand', 'spoiler', 'hate', 'need', 'mess', 'change', 'drop', 'date', 'netflix', 'yeah', 'daon', 'park', 'thank', 'lmao', 'damn', "i'll",
                   'kang', 'shinwoo', 'taekyung', 'mean', 'woman', 'hope', 'read', 'fact', 'opinion', 'stuff', 'feel', 'kdrama', 'talk', 'song', 'hype',
                   'title', 'type'))

# 사용 함수

In [35]:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('V'):
        return 'v'
    elif pos_tag.startswith('N'):
        return 'n'
    elif pos_tag.startswith('J'):
        return 'a'
    elif pos_tag.startswith('R'):
        return 'r'
    else:
        return None

# CustomTokenizer class setting

In [36]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, a):
        # a = ' '.join(a)
        word_tokens = self.tagger(a)
        
        words = []
        for i in word_tokens:
            text = re.sub('[^a-zA-Z0-9\']','',i).strip()
            text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`…》]','', text)
            if(text != ''):
                words.append(text)
        
        tag_words = nltk.pos_tag(words)
        pos_words = [word for word in tag_words if word[1][0] in {'N'}] #'V','N','J','R'

        temp_list = []
        for token, pos_tag in pos_words:
            tag = get_wordnet_pos(pos_tag)
            if tag != None:
                temp_list.append((token, get_wordnet_pos(pos_tag)))
        lemma = WordNetLemmatizer()
        token_final = [lemma.lemmatize(token, pos=tag) for token, tag in temp_list]
        long_words = [i for i in token_final if len(i) > 2]
        results = [w for w in long_words if w not in stop_words]
        
        return results

# 작업

In [37]:
def tfidf_vectorizing(reviews):
    ngram_range = (1,2)

    custom_tokenizer = CustomTokenizer(text_to_word_sequence)

    tfidf = TfidfVectorizer(tokenizer=custom_tokenizer,ngram_range = ngram_range,\
        stop_words = stop_words, max_df=10, min_df=2 ,max_features=5000).fit(reviews)
    candidates = tfidf.get_feature_names()
    
    return candidates

# 임베딩

In [38]:
def embedding(sentences,candidates):
    from sentence_transformers import SentenceTransformer

    model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

    doc_embedding = model.encode([sentences])
    candidate_embeddings = model.encode(candidates)
    # drama_embedding = model.encode(reviews_by_drama)

    return doc_embedding, candidate_embeddings

# 코사인 유사도 함수

In [39]:
from sklearn.metrics.pairwise import cosine_similarity
top_n = 500
diversity = 0.7

def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):
    word_doc_distances = cosine_similarity(candidate_embeddings,doc_embedding)
    word_distances = cosine_similarity(candidate_embeddings) 

    keywords_idx = [np.argmax(word_doc_distances)]
    candidates_idx = [i for i in range(len(candidates)) if i != keywords_idx[0]]
    
    for _ in range(top_n-1):
        try:
        # 후보 키워드들의 문서유사도 값
            candidate_similarities = word_doc_distances[candidates_idx, :]
            # 후보 키워드와 가장 유사한 키워드
            target_similarities = np.max(word_distances[candidates_idx][:,keywords_idx], axis=1) 

            mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1,1)
            mmr_idx = candidates_idx[np.argmax(mmr)]

            keywords_idx.append(mmr_idx)
            candidates_idx.remove(mmr_idx)
        except:
            break

    # 본 단어 임베딩벡터에서 추출키워드에 해당하는 벡터만
    keywords_vector = candidate_embeddings[keywords_idx][:] 
    keywords = [words[idx] for idx in keywords_idx]
    
    return keywords, keywords_vector


In [31]:
# for idx, drama in enumerate(drama_embedding):
#     keywords, keywords_vector = mmr(drama.reshape(1,-1) ,candidate_embeddings,candidates,top_n,diversity)
#     print(titles[idx])
#     print(keywords[:10],'\n')

### keyword Dataframe 생성

In [40]:
import pickle

# drama titles in csv
drama_titles = df.groupby('title',as_index=False).count().title

output = pd.DataFrame(None,columns=drama_titles)

top_n = 500
diversity = 0.6

for i,title in tqdm(enumerate(drama_titles)):
    reviews = df[df.title == title].reviews.to_list()

    # tfidf custom vectorizing
    candidates = tfidf_vectorizing(reviews) # reviews = list

    # embedding
    sentences = ','.join(reviews)
    doc_embedding, candidate_embeddings = embedding(sentences, candidates) 

    # keywords extraction

    keywords, keywords_vector = mmr(doc_embedding,candidate_embeddings,candidates,top_n,diversity)

    with open(f'drama_{i}.pickle', 'wb') as f:
        pickle.dump(keywords, f, pickle.HIGHEST_PROTOCOL)
        pickle.dump(keywords_vector, f, pickle.HIGHEST_PROTOCOL)

    output[title] = keywords[:7]
    print(title, keywords[:7])

output.to_csv('0506_theme_Keywords.csv')

1it [00:16, 16.81s/it]

18어게인 ['pain stunning', 'hyun actor', 'society mother', 'taste plot', 'husband lot', 'child dream', "episode's epilogue"]


2it [00:30, 15.00s/it]

365:운명을거스르는1년 ['character suspense', 'hyun joon', 'man chemistry', 'sweet loophole', 'universe travel', 'romance day', 'soundtrack']


3it [00:44, 14.62s/it]

...

한번다녀왔습니다 ['favourite family', 'noona', 'lee jung', 'acting cast', 'rollercoaster', 'sin', 'kid story']


91it [29:41, 21.17s/it]

해피니스 ['happiness zombie', 'problem couple', 'gym trainer', 'cure vaccine', 'character wise', 'chemistry villain', 'selfishness']


92it [30:13, 19.71s/it]

호텔델루나 ['love acting', 'lamb', 'wish hotel', 'luna fighting', 'story development', "jang's snark", 'alldramaaddict']


In [41]:
output.to_csv('0506_theme_Keywords.csv')

In [ ]:
# with open('title.pickle', 'rb') as f:
#     keywords = pickle.load(f)
#     keywords_vector = pickle.load(f)

# KMEANS - 작품별 연관키워드 뽑을 때 사용

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

X = keywords_vector

true_k = 6
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X.T)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = keywords # = candidates
for i in range(true_k):
    print("["),
    for ind in order_centroids[i, :20]:
        print(f'"{terms[ind]}"', end=",")
    print("],\n")

In [ ]:
x = list(range(1,11))
y = []
for i in x:
    model = KMeans(n_clusters=i, init='k-means++', max_iter=100, n_init=1)
    model.fit(X.T)
    y.append(model.inertia_)

import matplotlib.pyplot as plt

plt.plot(x,y,label='linear')
plt.show();